In [ ]:
%cd '/content/drive/MyDrive/popularity_pred'

/content/drive/MyDrive/popularity_pred


In [ ]:
!pip install scikit-optimize, xgboost, hyperopt

In [ ]:
from hyperopt import fmin, tpe, hp, anneal, Trials
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics
from numpy import sqrt
from numpy import argmax
from sklearn.metrics import classification_report

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from prettytable import PrettyTable
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from preprocess import label_encode, standard_scaler
from feature_gen import gen_text_features, gen_date_features
from sklearn.model_selection import StratifiedKFold
from evaluator import spearmanr, reg_eval_metrics, trmse, overall_eval_metric
from sklearn.metrics import confusion_matrix
import warnings
import xgboost as XGB
from sklearn.metrics import f1_score

In [ ]:
def cluster_data(df, num_clusters, start_day='Day01', end_day='Day30'):
    from sklearn import metrics
    from sklearn.cluster import KMeans, DBSCAN
    #X = df.loc[:, 'Day01':'Day30']
    X = df.loc[:, start_day:end_day]
    #X = df.loc[:, 'Day21':'Day30']
    for col in X.columns:
        X[col] = X[col]/df['Day30']
    X = X.values
    #X = np.log(X+ 0.001)

    kmeans = KMeans(n_clusters = num_clusters, init = 'k-means++', max_iter =300, n_init = 10, random_state = 0)
    kmeans.fit(X)
    labels = kmeans.labels_
    aucc = metrics.silhouette_score(X, labels, metric='euclidean')
    print ('wcss: ', kmeans.inertia_, 'silhouette score: ', aucc)

    return kmeans, labels

warnings.simplefilter("ignore")

users_df = pd.read_csv('data/users_TRAIN.csv')
image_info_df = pd.read_csv('data/img_info_TRAIN.csv')
headers_df = pd.read_csv('data/headers_TRAIN.csv')
popularity_df = pd.read_csv('data/popularity_TRAIN.csv')

cdf = pd.merge(headers_df, image_info_df, on='FlickrId')
combined_df = pd.merge(cdf, users_df, on='UserId' )
combined_df = pd.merge(combined_df, popularity_df, on='FlickrId')
print ("Final data: ", combined_df.shape)

#combined_df = combined_df.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
# Drop columns that have 80 per of Nan
#combined_df = combined_df.drop(['Camera', 'Latitude', 'Longitude', 'Country'], axis=1)

print (combined_df.shape)
combined_df = combined_df.drop_duplicates('URL')
print (combined_df.shape)

# Clustering
num_clusters = 2
kmeans_period_1, labels_period_1 = cluster_data(combined_df, num_clusters, 'Day01', 'Day10')
kmeans_period_2, labels_period_2 = cluster_data(combined_df, num_clusters, 'Day11', 'Day20')
kmeans_period_3, labels_period_3 = cluster_data(combined_df, num_clusters, 'Day21', 'Day30')

# Preprocessing data
# Data for cluster Prediction
X = combined_df.loc[:, 'FlickrId':'GroupsAvgPictures']
X['UserId'] = label_encode(X['UserId'])
X['Username'] = label_encode(X['Username'])

X = gen_text_features(X, 'Title')
#X = gen_text_features(X, 'URL')
X = gen_text_features(X, 'Description')
#X = gen_text_features(X, 'Country')
merge_tags = lambda x: ' '.join(x)
X['Tags'] = X['Tags'].map(merge_tags)
X = gen_text_features(X, 'Tags')

# Feature gen for date stamp
X = gen_date_features(X, 'DatePosted')
X = gen_date_features(X, 'DateTaken')
X = gen_date_features(X, 'DateCrawl')

X = X.drop(['URL', 'Title', 'Description', 'Tags', 'Country',
            'Unnamed: 0_y', 'Unnamed: 0_x',
            'Camera'], axis=1)
X = X.drop(['UserId', 'Username', 'FlickrId'], axis=1)
X = X.drop(['DatePosted', 'DateTaken', 'DateCrawl'], axis=1)
X = X.drop(['Latitude', 'Longitude'], axis=1)

X = X.values
X = standard_scaler(X)
n_splits = 3
skfolds = StratifiedKFold(n_splits=n_splits, random_state=42)

# unique, counts = np.unique(labels, return_counts=True)
# print (np.asarray((unique, counts)).T)

# Y = labels
Y_scale = combined_df['Day30'].values
Y_scale = np.log(np.log(Y_scale + 1) + 1)

# for train_index, test_index in skfolds.split(X, Y):
#     X_train = X[train_index]
#     y_train = Y[train_index]
#     X_val = X[test_index]
#     y_val = Y[test_index]
#     y_train_reg = Y_scale[train_index]
#     y_test_reg  = Y_scale[test_index]
#     break

In [ ]:
# period_1_train_indices, period_1_test_indices = [], []
# for train_index, test_index in skfolds.split(X, labels_period_1):
#     period_1_train_indices.append(train_index)
#     period_1_test_indices.append(test_index)

# X_train = X[period_1_train_indices[0]]
# X_val = X[period_1_test_indices[0]]

# y_train_reg = Y_scale[period_1_train_indices[0]]
# y_val_reg  = Y_scale[period_1_test_indices[0]]

# period 2
period_1_train_indices, period_1_test_indices = [], []
for train_index, test_index in skfolds.split(X, labels_period_2):
    period_1_train_indices.append(train_index)
    period_1_test_indices.append(test_index)

X_train = X[period_1_train_indices[0]]
X_val = X[period_1_test_indices[0]]

y_train_reg = Y_scale[period_1_train_indices[0]]
y_val_reg  = Y_scale[period_1_test_indices[0]]

In [ ]:
# Choose hyperparameter domain to search over
space = {
        'max_depth':hp.choice('max_depth', np.arange(10, 25, 1, dtype=int)),
        'n_estimators':hp.choice('n_estimators', np.arange(1000, 10000, 10, dtype=int)),
        'colsample_bytree':hp.quniform('colsample_bytree', 0.5, 1.0, 0.1),
        'min_child_weight':hp.choice('min_child_weight', np.arange(250, 350, 10, dtype=int)),
        'subsample':hp.quniform('subsample', 0.7, 0.9, 0.1),
        'eta':hp.quniform('eta', 0.1, 0.3, 0.1),

        'objective':'reg:squarederror',

        'tree_method':'gpu_hist',
        'eval_metric': 'rmse',
    }

def score(params):
    model = XGBRegressor(**params)

    model.fit(X_train, y_train_reg, eval_set=[(X_train, y_train_reg), (X_val, y_val_reg)], verbose=False, early_stopping_rounds=10)
    ypred = model.predict(X_val)

    (rmse, mae, r2) = reg_eval_metrics(y_val_reg, ypred)
    p = PrettyTable(['RMSE', 'MAE', 'R2', 'Spearmanr'])
    p.add_row([rmse, mae, r2, spearmanr(y_val_reg, ypred)])
    print(p)

    return {'loss': rmse, 'status': STATUS_OK}

def optimize(trials, space):

    best = fmin(score, space, algo=tpe.suggest, max_evals=1000)
    return best

trials = Trials()
best_params = optimize(trials, space)

# Return the best parameters
space_eval(space, best_params)

Streaming output truncated to the last 5000 lines.
|         RMSE        |         MAE         |         R2         |     Spearmanr      |
+---------------------+---------------------+--------------------+--------------------+
| 0.25993724193949713 | 0.20274176066140326 | 0.6235793306421025 | 0.7662376945899146 |
+---------------------+---------------------+--------------------+--------------------+
+---------------------+--------------------+--------------------+--------------------+
|         RMSE        |        MAE         |         R2         |     Spearmanr      |
+---------------------+--------------------+--------------------+--------------------+
| 0.26112552787802157 | 0.2037038580215047 | 0.6201298997280102 | 0.7639311375385789 |
+---------------------+--------------------+--------------------+--------------------+
+--------------------+---------------------+--------------------+--------------------+
|        RMSE        |         MAE         |         R2         |     Spear

{'colsample_bytree': 0.6000000000000001,
 'eta': 0.2,
 'eval_metric': 'rmse',
 'max_depth': 13,
 'min_child_weight': 320,
 'n_estimators': 1840,
 'objective': 'reg:squarederror',
 'subsample': 0.8,
 'tree_method': 'gpu_hist'}

In [ ]:
print('period 2')
print(best_params)
print()
space_eval(space, best_params)

period 2
{'colsample_bytree': 0.6000000000000001, 'eta': 0.2, 'max_depth': 3, 'min_child_weight': 7, 'n_estimators': 84, 'subsample': 0.8}



{'colsample_bytree': 0.6000000000000001,
 'eta': 0.2,
 'eval_metric': 'rmse',
 'max_depth': 13,
 'min_child_weight': 320,
 'n_estimators': 1840,
 'objective': 'reg:squarederror',
 'subsample': 0.8,
 'tree_method': 'gpu_hist'}

In [ ]:
print('period 1')
print(best_params)
print()
space_eval(space, best_params)

period 1
{'colsample_bytree': 0.9, 'eta': 0.2, 'max_depth': 10, 'min_child_weight': 7, 'n_estimators': 49, 'subsample': 0.7000000000000001}



{'colsample_bytree': 0.9,
 'eta': 0.2,
 'eval_metric': 'rmse',
 'max_depth': 20,
 'min_child_weight': 320,
 'n_estimators': 1490,
 'objective': 'reg:squarederror',
 'subsample': 0.7000000000000001,
 'tree_method': 'gpu_hist'}